In [7]:
import pandas as pd
import graphviz
import uuid


class Gini():
    def __init__(self, file) -> None:
        self.df = pd.read_csv(file)
        self.df = pd.DataFrame(self.df)
        self.seuils_optimaux = [] # arbre contenant les informations : les seuils optimaux, avec leur gini, la variable, les ID de noeuds correspondant, le niveau dans l'arbre
        self.tree = {} # arbre qui servira pour l'affichage, dictionnaire stockant des noeuds, leurs noeuds fils et les modalités de ces noeuds fils
        self.main()

    def division(self, df):
        """
            Renvoyer le seuil optimal, sa variable et son indice de Gini pour un dataframe donné
        """
        # stocker les valeurs des indices de Gini, leurs seuils et variables correspondantes
        tab_gini = pd.DataFrame(columns=['variable', 'seuil', 'gini'])

        # calcul de Dk
        p1 = df['Y'].value_counts()['A']  # nombre de modalités A dans ce noeud
        p2 = df['Y'].value_counts()['B']
        n = p1 + p2  # nombre d'individus
        Dk = 1 - ((p1/n)**2 + (p2/n)**2)  # indice de Gini avant séparation

        # parcourir chaque variable X1, X2...
        i = 0
        for x in df.columns[1:]:
            # parcourir chaque valeur de cette variable comme seuil
            for seuil in df[x]:
                # liste des modalités du noeud de gauche respectant le seuil
                noeud_gauche = df[df[x] <= seuil].Y
                noeud_droite = df[df[x] > seuil].Y

                # calculer l'indice de gini seulement lorsque le noeud n'est pas totalement pur (contient des modalités différentes)
                Dkg, Dkd = 0, 0
                if 'A' in noeud_gauche.tolist() and 'B' in noeud_gauche.tolist():
                    p1g = noeud_gauche.value_counts().A
                    p2g = noeud_gauche.value_counts().B
                    ng = p1g + p2g
                    Dkg = (1 - ((p1g/ng)**2 + (p2g/ng)**2)) * \
                        (ng/n)  # indice de Gini de ce noeud

                if 'A' in noeud_droite.tolist() and 'B' in noeud_droite.tolist():
                    p1d = noeud_droite.value_counts().A
                    p2d = noeud_droite.value_counts().B
                    nd = p1d + p2d
                    Dkd = (1 - ((p1d/nd)**2 + (p2d/nd)**2)) * (nd/n)

                gini = Dk - (Dkg + Dkd)  # indice de Gini global
                tab_gini.loc[i] = [x, seuil, gini]  # ajout des informations
                i += 1

        tab_gini.sort_values(by="gini", ascending=False, inplace=True)
        tab_gini.reset_index(drop=True, inplace=True)
        return tab_gini.iloc[0] # renvoyer le meilleur indice de gini, la variable et le seuil correspondants

    def recursion(self, df, level=0, parent_noeud_id=None):
        """
            Construire un arbre en séparant les modalités récursivement à l'aide de la méthode division
            
            Exemple : 2: {'parent': 0, 'enfant': [3, 'd'], 'mg': ['A', 'A', 'A'], 'md': ['A', 'B']
            La clé 2 de l'arbre correspond au noeud d'identifiant 2, ayant pour enfant
            droit le noeud d'ID 3 qui a des modalités différentes ['A','B']
        """
        result = self.division(df) # obtenir les infos de la meilleure division pour ce noeud

        # générer un ID pour ce noeud, il y a autant d'ID de noeuds (impurs) que de seuils optimaux
        noeud_id = len(self.seuils_optimaux) 
        
        # pour ce noeud, ajout du seuil optimal dans la liste avec le resultat de division
        self.seuils_optimaux.append([level, result['variable'], result['seuil'], result['gini'], noeud_id])

        # modalités à gauche et à droite répartis avec le seuil
        mg = list(df[df[result['variable']] <= result['seuil']].Y)
        md = list(df[df[result['variable']] > result['seuil']].Y)
        
        # construction de l'arbre : ajout du noeud_id, son parent
        # ses enfants (ceux ayant modalités différentes), et modalités de l'enfant gauche, et droite
        self.tree[noeud_id] = {'parent': parent_noeud_id, 'enfant': [], 'mg': mg, 'md': md}

        # dans l'arbre, ajouter l'ID de ce noeud à la liste des enfants de son parent s'il en a un
        # préciser si l'enfant est à gauche ou à droite
        if parent_noeud_id is not None:
            if self.tree[parent_noeud_id]['mg'] == self.tree[noeud_id]['mg'] + self.tree[noeud_id]['md'] \
            or self.tree[parent_noeud_id]['mg'] == self.tree[noeud_id]['md'] + self.tree[noeud_id]['mg'] :
                self.tree[parent_noeud_id]['enfant'].append((noeud_id, 'g'))
            else:
                self.tree[parent_noeud_id]['enfant'].append((noeud_id, 'd'))

        # appel récursif de cette fonction sur les sous-ensembles de données gauche et droite s'ils contiennent des modalités différentes
        for c in ['g', 'd']:
            if 'A' in self.tree[noeud_id][f'm{c}'] and 'B' in self.tree[noeud_id][f'm{c}']:
                new_df = df[df[result['variable']] <= result['seuil']
                            ] if c == 'g' else df[df[result['variable']] > result['seuil']]
                self.recursion(new_df, level + 1, noeud_id)

    def display(self):
        """
            Afficher l'arbre avec graphviz en utilisant seuils_optimaux et tree

            Dans l'image générée :
            - Les noeuds impurs seront affichés avec l'ID et l'indice de Gini,
                puis deux noeuds fils pour la séparation seront créés.
            - Les noeuds purs afficheront leur modalité.
            - Les branches afficheront les informations sur les variables et les seuils.
        """
        dot = graphviz.Digraph(comment='Arbre de décision',
                            graph_attr={'size': '20,20!', 'fontsize': '14'})

        for noeud in self.seuils_optimaux:
            level, variable, seuil, gini, noeud_id = noeud
            mg = self.tree[noeud_id]['mg']
            md = self.tree[noeud_id]['md']

            nb_A_gauche = mg.count('A')
            nb_B_gauche = mg.count('B')
            nb_A_droite = md.count('A')
            nb_B_droite = md.count('B')
            if level > 0:


                if 'A' in mg and 'B' in mg:
                    mg = None
                if 'A' in md and 'B' in md:
                    md = None

                if mg:
                    gauche_id = str(uuid.uuid4())
                    dot.node(gauche_id, label=f"A: {nb_A_gauche}\nB: {nb_B_gauche}", shape="oval")
                    dot.edge(f"{noeud_id}", gauche_id, label=f"{variable} ≤ {seuil}")
                if md:
                    droite_id = str(uuid.uuid4())
                    dot.node(droite_id, label=f"A: {nb_A_droite}\nB: {nb_B_droite}", shape="oval")
                    dot.edge(f"{noeud_id}", droite_id, label=f"{variable} > {seuil}")

            if level == 0:
                dot.node(f"{noeud_id}", label=f"ID {noeud_id}\nG = {gini:.4f}\nA: {nb_A_gauche + nb_A_droite}\nB: {nb_B_gauche + nb_B_droite}", shape="oval")
            else:
                if mg is None and md is None:
                    continue

                dot.node(f"{noeud_id}", label=f"ID {noeud_id}\nG = {gini:.4f}\nA: {nb_A_gauche + nb_A_droite}\nB: {nb_B_gauche + nb_B_droite}", shape="oval")
                parent_id = self.tree[noeud_id]['parent']
                parent_node = [n for n in self.seuils_optimaux if n[-1] == parent_id][0]
                _, parent_variable, parent_seuil, _, _ = parent_node

                direction = ''
                parent_enfant = self.tree[parent_id]['enfant']
                est_gauche = any([enfant for enfant in parent_enfant if enfant[0] == noeud_id and enfant[1] == 'g'])

                if est_gauche:
                    direction = '≤'
                else:
                    direction = '>'
                dot.edge(f"{parent_id}", f"{noeud_id}", label=f"{parent_variable} {direction} {parent_seuil}")

        dot.render("arbre_decision", view=True, format="png")


    def main(self):
        self.recursion(self.df)
        self.display()
        print(self.tree)


if __name__ == "__main__":
    Gini('data.csv')


{0: {'parent': None, 'enfant': [(1, 'g'), (2, 'd')], 'mg': ['A', 'B', 'B', 'B'], 'md': ['A', 'A', 'A', 'A', 'B']}, 1: {'parent': 0, 'enfant': [], 'mg': ['A'], 'md': ['B', 'B', 'B']}, 2: {'parent': 0, 'enfant': [(3, 'd')], 'mg': ['A', 'A', 'A'], 'md': ['A', 'B']}, 3: {'parent': 2, 'enfant': [], 'mg': ['B'], 'md': ['A']}}
